In [1]:
import numpy as np
import math


In [2]:
def area_flow(Dia):
    A_cross = math.pi * (Dia / 2) ** 2
    return A_cross



In [3]:
def get_LMTD_counter(T_hot_in, T_cold_out, T_hot_out, T_cold_in):
    # Log Mean Temperature Difference (LMTD) for counter-flow
    delta_T1 = T_hot_in - T_cold_out
    delta_T2 = T_hot_out - T_cold_in
    
    if abs(delta_T1 - delta_T2) < 1e-10:
        LMTD = delta_T1  # Avoid division by zero when delta_T1 = delta_T2
    else:
        LMTD = (delta_T1 - delta_T2) / math.log(delta_T1 / delta_T2)

    return LMTD

In [4]:
def get_mass_flow(rho, velocity,A_cross):
    m_dot = rho * velocity * A_cross
    return m_dot


In [5]:
def get_Q(mass_flow, cp, temp_diff):
    Q = mass_flow*cp*temp_diff
    return Q

In [6]:
def get_h(cp,mu,m_dot,km,dia,kf):
    Prandtl_number = cp*mu/km 
    reynolds = (4*m_dot)/(math.pi*dia*mu)
    h = 0.023 * reynolds**0.8 * Prandtl_number**0.4 * (kf/dia)

    return h



In [10]:
def get_LMTD_parra(T_hot_in, T_cold_out, T_hot_out, T_cold_in):
    # Log Mean Temperature Difference (LMTD) for counter-flow
    delta_T1 = T_hot_in - T_cold_in
    delta_T2 = T_hot_out - T_cold_out
    
    if abs(delta_T2 - delta_T1) < 1e-10:
        LMTD = delta_T2  # Avoid division by zero when delta_T1 = delta_T2
    else:
        LMTD = (delta_T2 - delta_T1) / math.log(delta_T2 / delta_T1)

    return LMTD

In [ ]:
#Find Qc for testing toy model

Cp_c = 4180 #J/kg K
m_dot_c = 0.2 #kg/s
T_co = 60 #c
T_ci = 25 #c

dia = 0.008 #m thin wall so dia outter is same as d inner

Qc = get_Q(m_dot_c,Cp_c,(T_co - T_ci))
print(Qc) #w

#Lets find T_ho
Cp_h = 4310 #J/kg K
m_dot_h = 0.3 #kg/s
T_hi = 140 #c

T_ho = T_hi - Qc/(m_dot_h * Cp_h)
print(T_ho)

LMTD_para = get_LMTD_parra(T_hi,T_co,T_ho,T_ci)
print(LMTD_para)

U = 550 #w/m2 K
surface_area = Qc/(U * LMTD_para) #m2
print(surface_area)

Length =  surface_area/(math.pi*dia)
print(Length)

29260.0
117.37045630317093
82.87219381751132
0.6419523551789764
25.542472639054544


In [45]:
#Current purifier

Cp = 5188 #J/kg K
v_dot = 3.5 #cuft
T_co = 80 #k
T_hi = 295 #k
T_ho = 273 #k guess

rho_25 = 0.1607 #kg/m3
rho_cold = 0.622 #kg/m3

#unit conversion
L_to_m = 0.001 #m3
cuft_to_l = 28.3168 #l
cuft_to_m = cuft_to_l*L_to_m
in_to_m = 0.0254

rho_25 = rho_25*cuft_to_m
m_dot = v_dot*rho_25/60 #kg/s

#==-=--=-=-=-=-=-==-=-=-=-=--=-=-=-=-==-==-=-=-=-=-==-=-

Qh = abs(get_Q(m_dot,Cp,(T_ho - T_hi)))

T_ci = T_co - Qh/(m_dot * Cp)
print(f"Inside cold temp is: {T_ci}")

LMTD_counter = get_LMTD_counter(T_hi,T_co,T_ho,T_ci)
print(f"Log mean temp is: {LMTD_para}")

#Getting h
mu = 15e-6 #pa
kf = .1
km = 15

#We have two sections bed and upper
dia1 = .5*in_to_m
h1 = get_h(Cp,mu,m_dot,km, dia1,kf)
dia2 = (3/4)*in_to_m
h2 = get_h(Cp,mu,m_dot,km, dia2,kf)
U = (1/(h1*math.pi*dia1))+(1/(h2*math.pi*dia2))+math.log(dia2/dia1)/(2*math.pi*km) #w/m2 K
print(U)
surface_area = Qh/(U * LMTD_counter) #m2
print(surface_area)

Length =  surface_area/(math.pi*dia)
print(Length) #m

#We know the length is 28" 
L = 28*in_to_m
surface_area = math.pi*(dia2-dia1)*L
Q = U*surface_area*LMTD_counter
print(f"Q is {Q}")

Qc = get_Q(m_dot,Cp,(T_hi - T_co))
print(Qc)
print(Qc/Q)

Inside cold temp is: 58.0
Log mean temp is: 220.20472265400133
6.8132852654802045
0.02068256794287754
0.013166931695771735
Q is 20.783102459486464
296.08422646245333
14.246392088939803


In [30]:
def get_partial_pressure(mole_fraction, P_sat):
    partial_pressure = mole_fraction*P_sat/100
    return partial_pressure

In [61]:
mole_fraction_N2 = 1 # % precentage
mole_fraction_O2 = 1 # precentage

Temp = 77 #K
P_sat_N2 = 1 # bar
P_sat_O2 = .2 #bar

operating_pressure = 0.689476 #bar
flow_rate = 2.7 #nm3/h equaled to 45 l per min



partial_N2 = get_partial_pressure(mole_fraction_N2,P_sat_N2)
partial_O2 = get_partial_pressure(mole_fraction_O2,P_sat_O2)
print(f"partial P N2, {partial_N2} and partial P O2, {partial_O2}.")
partial_air = partial_N2 + partial_O2
print(f"partial P air: {partial_air}")

mole_fraction_air = (partial_air/operating_pressure)
print(f"Mole fraction of air is {mole_fraction_air}%")

t = 25 #hours
nitrogen_adsorbed = mole_fraction_air*flow_rate*t
print(f"Nitrogen needed to be adsorbed is: {nitrogen_adsorbed}")
v_a = nitrogen_adsorbed*1000000 #cc
energy_of_adsorption = 300.2 #k
c = math.exp(energy_of_adsorption/Temp)
vm = 181 #might not be true? units g
constant = (1+c*(operating_pressure/P_sat_N2))/(c*(operating_pressure/P_sat_N2))
mass_of_adsorpant = (v_a/vm)*constant
print(f"The mass of the adsorpant is {mass_of_adsorpant} grams.")


#Sizing of bed

g_to_kg = 0.001
density_carbon = 0.44 # g/cc
volume_carbon = mass_of_adsorpant/(density_carbon*g_to_kg)
cc_to_cuft = 3.5315e-5 # cuft
volume_carbon = volume_carbon*cc_to_cuft
print(volume_carbon)

R = 1 #L/D this is based on the velocity of the gas!
cuft_to_cuin = 1728
Dia = 12#(4*volume_carbon*cuft_to_cuin/math.pi*R)**(1/3)
L = ((4*volume_carbon*cuft_to_cuin*R**2)/math.pi)**(1/3)

print(L,Dia)


partial P N2, 0.01 and partial P O2, 0.002.
partial P air: 0.012
Mole fraction of air is 0.017404521694736292%
Nitrogen needed to be adsorbed is: 1.1748052143946999
The mass of the adsorpant is 6681.438874833401 grams.
536.26139514714
105.66799415530265 12


In [ ]:
#How much adsorbant do we need?
break_through_time = 30 #days
day_to_sec = 86400 #s
g_to_kg = 0.001
Mass_flow_rate_N2 = m_dot*mole_fraction_N2
print(Mass_flow_rate_N2)
Nitrogen_required_to_be_adsorbed = break_through_time*day_to_sec*Mass_flow_rate_N2 #gN2
print(Nitrogen_required_to_be_adsorbed*g_to_kg)

uptake_carbon = 195.2 #cc/kgC
density_n2 = 794 #kg/m3 at 80k
density_carbon = 0.44 # g/cc
m3_to_cm3 = 1/1000000

#Using the above calculation
mass_of_carbon = (Nitrogen_required_to_be_adsorbed*g_to_kg)/(density_n2*uptake_carbon*m3_to_cm3)
print(mass_of_carbon)

volume_carbon = mass_of_carbon/(density_carbon*g_to_kg)
cc_to_cuft = 3.5315e-5 # cuft
volume_carbon = volume_carbon*cc_to_cuft
print(volume_carbon)

#Sizing of bed
R = 10 #L/D this is based on the velocity of the gas!
cuft_to_cuin = 1728
Dia = (4*volume_carbon*cuft_to_cuin/math.pi*R)**(1/3)
L = ((4*volume_carbon*cuft_to_cuin*R**2)/math.pi)**(1/3)

print(Dia,L) #inches


0.00026544640266666663
0.6880370757119999
4.439269648593963
0.3563018355456723
19.865094657818588 42.79804905157142
